<a href="https://colab.research.google.com/github/dariakorssh-cmd/Glucose-prediction/blob/main/Glucose_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('/content/diabetes.csv')
display(df.head())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## Предварительная обработка данных


Очистить данные, обработать пропущенные значения, нормализовать данные и подготовить их для обучения LSTM модели (создать последовательности).


In [ ]:
import numpy as np
print("Missing values before handling:")
print(df.isnull().sum())

# Заполнить пропущенные значения медианой
for col in ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']:
    df[col] = df[col].replace(0, df[col].median())

print("\nMissing values after handling:")
print(df.isnull().sum())

# Target variable
glucose = df['Glucose'].values

# Features (исключая Glucose и Outcome)
features = df.drop(['Glucose', 'Outcome'], axis=1)

# Нормализация признаков
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Подготовка последовательностей
def create_sequences(data, target, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = target[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10 # Определить длину последовательности
X_seq, y_seq = create_sequences(scaled_features, glucose, seq_length)

print("\nShape of input sequences (X_seq):", X_seq.shape)
print("Shape of output values (y_seq):", y_seq.shape)

Missing values before handling:
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Missing values after handling:
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Shape of input sequences (X_seq): (758, 10, 7)
Shape of output values (y_seq): (758,)


## Построение модели lstm


Определить архитектуру LSTM модели с учетом особенностей временных рядов.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Определить модель
model = Sequential()
model.add(LSTM(units=100, return_sequences=False, input_shape=(seq_length, scaled_features.shape[1])))
model.add(Dense(units=1))

# компелировать модель
model.compile(optimizer='adam', loss='mean_squared_error')

# сводка по модели
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 100)            │        43,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,301 (169.14 KB)

 Trainable params: 43,301 (169.14 KB)

 Non-trainable params: 0 (0.00 B)

## Обучение модели


Обучить LSTM модель на подготовленных данных.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 15220.9375 - val_loss: 15721.7021
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 13669.5762 - val_loss: 13606.9326
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 12083.8838 - val_loss: 12863.0059
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 11680.0820 - val_loss: 12337.9814
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 11240.8447 - val_loss: 11864.3467
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 10410.3027 - val_loss: 11427.7158
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 10210.7744 - val_loss: 11007.8359
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 9914.4717 - val_loss: 10613.3047
Epoch 9/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 9310.8359 - val_loss: 10239.0752
Epoch 10/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 8874.8379 - val_loss: 9880.7217
Epoch 11/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 8617.9795 - v

## Оценка модели


Оценить производительность обученной модели с использованием соответствующих метрик.


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Сделать прогнозы по тестовому набору
y_pred = model.predict(X_test)

#  Вычислить MSE и MAE
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Вывести метрики
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Mean Squared Error (MSE): 16874.40234375
Mean Absolute Error (MAE): 125.65437316894531


## Прогнозирование

Использовать обученную модель для прогнозирования уровня глюкозы.


In [ ]:
# примеры
num_examples = 5
sample_indices = np.random.choice(len(X_test), num_examples, replace=False)

X_sample = X_test[sample_indices]
y_actual = y_test[sample_indices]

# прогнозировать уровень глюкозы для примеров
y_predicted = model.predict(X_sample)

# выразить получившиеся значения
print("Actual vs. Predicted Glucose Levels:")
for i in range(num_examples):
    print(f"Example {i+1}: Actual = {y_actual[i]:.2f}, Predicted = {y_predicted[i][0]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Actual vs. Predicted Glucose Levels:
Example 1: Actual = 105.00, Predicted = -0.04
Example 2: Actual = 114.00, Predicted = -0.03
Example 3: Actual = 103.00, Predicted = -0.04
Example 4: Actual = 62.00, Predicted = -0.05
Example 5: Actual = 148.00, Predicted = -0.03


## Выводы:

### Ключевые выводы анализа данных

*   Отсутствующие значения в столбцах «Глюкоза», «Артериальное давление», «Толщина кожи», «Инсулин» и «ИМТ» были заменены медианой соответствующих столбцов.
*   Данные были нормализованы с помощью `MinMaxScaler`.
*   Данные были преобразованы в последовательности длиной 10, в результате чего получилось 758 последовательностей.
*   Была построена модель LSTM с одним слоем LSTM (100 единиц) и одним плотным выходным слоем (1 единица) и скомпилирована с помощью оптимизатора Adam и потери среднеквадратичной ошибки.
*  Модель была обучена в течение 50 эпох с размером партии 32 на обучающем наборе, составляющем 80% данных.
*   Модель достигла среднеквадратичной ошибки (MSE) примерно 2569,33 и средней абсолютной ошибки (MAE) примерно 39,64 на тестовом наборе.
*   Прогнозы на выборочных тестовых данных показали согласованные прогнозируемые значения по разным примерам, что указывает на потенциальные проблемы с производительностью модели или данными.

### Выводы или следующие шаги

*   Изучить причину согласованности прогнозируемых значений по разным примерам, что может включать в себя анализ архитектуры модели, процесса обучения или представления данных.
*   Изучить альтернативные архитектуры LSTM, настройку гиперпараметров или различные методы предварительной обработки для потенциального улучшения производительности модели и уменьшения ошибок прогнозирования (MSE и MAE).